In [22]:
import cv2
import dlib
import numpy as np

# Load and resize the image
img = cv2.imread('/content/kid.jpg')
img = cv2.resize(img, (720, 640))
frame = img.copy()

# ---------- Model for Age Detection ----------
age_weights = "/path/to/your/Models/age_deploy.prototxt"
age_config = "https://github.com/eveningglow/age-and-gender-classification/blob/5b60d9f8a8608cdbbcdaaa39bf28f351e8d8553b/model/age_net.caffemodel"

try:
    age_Net = cv2.dnn.readNet(age_config, age_weights)
    model_loaded = True
except cv2.error as e:
    print(f"Error loading model: {e}")
    model_loaded = False

if model_loaded:
    # Model requirements for image
    ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)',
               '(25-32)', '(38-43)', '(48-53)', '(60-100)']
    model_mean = (78.4263377603, 87.7689143744, 114.895847746)

    # Storing the image dimensions
    fH, fW = img.shape[:2]
    Boxes = []  # To store the face coordinates
    mssg = 'Face Detected'  # Message to display on the image

    # ---------- Model for Face Detection ----------
    face_detector = dlib.get_frontal_face_detector()

    # Convert to grayscale
    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # ---------- Detecting Faces ----------
    faces = face_detector(img_gray)

    # If no faces are detected
    if not faces:
        mssg = 'No face detected'
        cv2.putText(img, mssg, (40, 40), cv2.FONT_HERSHEY_SIMPLEX, 2, (200), 2)
    else:
        # --------- Bounding Faces ---------
        for face in faces:
            x, y, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
            box = [x, y, x2, y2]
            Boxes.append(box)
            cv2.rectangle(frame, (x, y), (x2, y2), (0, 200, 200), 2)

        for box in Boxes:
            face = frame[box[1]:box[3], box[0]:box[2]]

            # ----- Image Preprocessing -----
            blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), model_mean, swapRB=False)

            # ------- Age Prediction -------
            age_Net.setInput(blob)
            age_preds = age_Net.forward()
            age = ageList[age_preds[0].argmax()]

            cv2.putText(frame, f'{mssg}: {age}', (box[0], box[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)

    # Display the image
    cv2.imshow("Detecting Age", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Model could not be loaded. Please check the paths and ensure the model files are correct.")


Error loading model: OpenCV(4.10.0) /io/opencv/modules/dnn/src/caffe/caffe_io.cpp:1126: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "/path/to/your/Models/age_deploy.prototxt" in function 'ReadProtoFromTextFile'

Model could not be loaded. Please check the paths and ensure the model files are correct.
